1. Learning sgRNA predictors from empirical data
    * Load scripts and empirical data
    * Generate TSS annotation using FANTOM dataset
    * Calculate parameters for empirical sgRNAs
    * Fit parameters
2. Applying machine learning model to predict sgRNA activity
    * Find all sgRNAs in genomic regions of interest 
    * Predicting sgRNA activity
3. Construct sgRNA libraries
    * Score sgRNAs for off-target potential
* Pick the top sgRNAs for a library, given predicted activity scores and off-target filtering
* Design negative controls matching the base composition of the library
* Finalizing library design

Make sure to run `jupyter notebook` after running `source activate CRISPRiaDesign`

# 1. Learning sgRNA predictors from empirical data
## Load scripts and empirical data

In [188]:
%run sgRNA_learning.py

In [2]:
genomeDict = loadGenomeAsDict(os.path.expanduser('~/data/reference-data-management/genomes/hg19.fa'))

Loading genome file...Done


In [1]:
genomeDict

NameError: name 'genomeDict' is not defined

In [163]:
#to use pre-calculated sgRNA activity score data (e.g. provided CRISPRi training data), load the following:
#CRISPRa activity score data also included in data_files
libraryTable_training = pd.read_csv('data_files/CRISPRi_trainingdata_libraryTable.txt', sep='\t', index_col = 0)
libraryTable_training.head()

,sublibrary,gene,transcripts,sequence
sgId,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GCCCCAGGATCAGGCCCCGCG
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GGCCGCCCTCGGAGAGCTCTG
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GACGGCGACCCTAGGAGAGGT
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GGTGCAGCGGGCCCTTGGCGG
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,drug_targets+kinase_phosphatase,AARS,all,GCGCTCTGATTGGACGGAGCG


In [4]:
sgInfoTable_training = pd.read_csv('data_files/CRISPRi_trainingdata_sgRNAInfoTable.txt', sep='\t', index_col=0)
sgInfoTable_training.head()

,Sublibrary,gene_name,length,pam coordinate,pass_score,position,strand,transcript_list
sgId,,,,,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323216,e39m1,70323216,+,['all']
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323296,e39m1,70323296,+,['all']
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323318,e39m1,70323318,+,['all']
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323362,e39m1,70323362,+,['all']
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,Drug_Targets+Kinase_Phosphatase,AARS,24,70323441,e39m1,70323441,+,['all']


In [5]:
activityScores = pd.read_csv('data_files/CRISPRi_trainingdata_activityScores.txt',sep='\t',index_col=0, header=None).iloc[:,0]
activityScores.head()

0
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1    0.348892
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1    0.912409
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1    0.997242
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1    0.962154
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1    0.019320
Name: 1, dtype: float64

In [6]:
tssTable = pd.read_csv('data_files/human_tssTable.txt',sep='\t', index_col=range(2))
tssTable.head()

position strand  \
gene  transcripts                                                             
A1BG  all                                                 58864864.0      -   
A1CF  ENST00000373993.1                                   52619744.0      -   
      ENST00000374001.2,ENST00000373997.3,ENST0000028...  52645434.0      -   
A2M   all                                                  9268752.0      -   
A2ML1 all                                                  8975067.0      +   

                                                         chromosome  \
gene  transcripts                                                     
A1BG  all                                                     chr19   
A1CF  ENST00000373993.1                                       chr10   
      ENST00000374001.2,ENST00000373997.3,ENST0000028...      chr10   
A2M   all                                                     chr12   
A2ML1 all                                                     chr12   

                                                                                           cage peak ranges  
gene  transcripts                                                                                            
A1BG  all                                                      [(58864822, 58864847), (58864848, 58864868)]  
A1CF  ENST00000373993.1                                                                                  []  
      ENST00000374001.2,ENST00000373997.3,ENST0000028...       [(52645379, 52645393), (52645416, 52645444)]  
A2M   all                                                 [(9268547, 9268556), (9268559, 9268568), (9268...  
A2ML1 all                                                 [(8975061, 8975072), (8975101, 8975108), (8975...

In [7]:
p1p2Table = pd.read_csv('data_files/human_p1p2Table.txt',sep='\t', header=0, index_col=range(2))
p1p2Table['primary TSS'] = p1p2Table['primary TSS'].apply(lambda tupString: (int(tupString.strip('()').split(', ')[0].split('.')[0]), int(tupString.strip('()').split(', ')[1].split('.')[0])))
p1p2Table['secondary TSS'] = p1p2Table['secondary TSS'].apply(lambda tupString: (int(tupString.strip('()').split(', ')[0].split('.')[0]),int(tupString.strip('()').split(', ')[1].split('.')[0])))
p1p2Table.head()

chromosome strand           TSS source           primary TSS  \
gene  transcript                                                                
A1BG  P1              chr19      -  CAGE, matched peaks  (58858938, 58859039)   
      P2              chr19      -  CAGE, matched peaks  (58864822, 58864847)   
A1CF  P1P2            chr10      -  CAGE, matched peaks  (52645379, 52645393)   
A2M   P1P2            chr12      -  CAGE, matched peaks    (9268507, 9268523)   
A2ML1 P1P2            chr12      +  CAGE, matched peaks    (8975206, 8975223)   

                         secondary TSS  
gene  transcript                        
A1BG  P1          (58858938, 58859039)  
      P2          (58864822, 58864847)  
A1CF  P1P2        (52645379, 52645393)  
A2M   P1P2          (9268528, 9268542)  
A2ML1 P1P2          (8975144, 8975169)

## Calculate parameters for empirical sgRNAs

### Because scikit-learn currently does not support any robust method for saving and re-loading the machine learning model, the best strategy is to simply re-learn the model from the training data

The above statement is untrue, pickle is often used to same models.

In [8]:
#Load bigwig files for any chromatin data of interest
bwhandleDict = {'dnase':BigWigFile(open('large_data_files/wgEncodeOpenChromDnaseK562BaseOverlapSignalV2.bigWig')),
'faire':BigWigFile(open('large_data_files/wgEncodeOpenChromFaireK562Sig.bigWig')),
'mnase':BigWigFile(open('large_data_files/wgEncodeSydhNsomeK562Sig.bigWig'))}

#### Run generateTypicalParamTable manually

In [29]:
libraryTable = libraryTable_training
sgInfoTable = sgInfoTable_training
bwFileHandleDict = bwhandleDict

In [13]:
lengthSeries = generateSgrnaLengthSeries(libraryTable)


In [16]:
lengthSeries.head()

sgId
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1    21
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1    21
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1    21
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1    21
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1    21
Name: length, dtype: int64

In [14]:
sgrnaPositionTable_p1p2 = generateSgrnaDistanceTable_p1p2Strategy(sgInfoTable,
                                                                  libraryTable,
                                                                  p1p2Table,
                                                                  False)


In [17]:
sgrnaPositionTable_p1p2.head()


,gene,transcript,primary TSS-Up,primary TSS-Down,secondary TSS-Up,secondary TSS-Down
sgId,,,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,AARS,P1P2,205,167,247,206
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,AARS,P1P2,125,87,167,126
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,AARS,P1P2,103,65,145,104
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,AARS,P1P2,59,21,101,60
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,AARS,P1P2,-20,-58,22,-19


In [15]:
baseTable, strand = generateRelativeBasesAndStrand(sgInfoTable,
                                                   tssTable,
                                                   libraryTable, genomeDict)


In [18]:
baseTable.head()

,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,...,0,1,2,3,4,5,6,7,8,9
sgId,,,,,,,,,,,,,,,,,,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,G,C,A,G,T,G,G,G,C,C,...,G,C,C,T,G,G,A,G,A,A
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,A,G,A,G,G,T,A,G,G,C,...,G,T,G,C,T,A,G,G,A,G
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,A,G,C,T,G,G,G,G,A,C,...,G,C,C,G,C,C,C,T,C,G
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,G,G,G,A,A,T,A,G,G,T,...,G,A,C,T,C,T,G,A,G,G
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,C,G,T,C,G,G,C,G,C,G,...,G,C,G,G,G,T,T,G,T,G


In [20]:
strand.head(20)

,same strand
sgId,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323466.24-all~e39m1,False
Drug_Targets+Kinase_Phosphatase=AARS_-_70323225.22-all~e39m1,True
Drug_Targets+Kinase_Phosphatase=AARS_-_70323276.23-all~e39m1,True
Drug_Targets+Kinase_Phosphatase=AARS_-_70323332.23-all~e39m1,True


In [21]:
booleanBaseTable = generateBooleanBaseTable(baseTable)


In [23]:
booleanBaseTable.head()

A                \
                                                      -30    -29    -28   
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  False   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...   True  False   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...   True  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...  False  False  False   

                                                                         \
                                                      -27    -26    -25   
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...   True   True  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...  False  False  False   

                                                                         \
                                                      -24    -23    -22   
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...   True  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...  False  False   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...   True  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...  False  False  False   

                                                           ...      T         \
                                                      -21  ...     0      1    
sgId                                                       ...                 
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  ...  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...  False  ...  False   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...  False  ...  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...  False  ...  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...  False  ...  False  False   

                                                                         \
                                                       2      3      4    
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False   True  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...  False  False   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...  False   True  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...  False  False  False   

                                                                         \
                                                       5      6      7    
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...  False  False   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...   True  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...   True   True  False   

                                                                  
                                                       8      9   
sgId                                                              
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  False  
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...  False  False  
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...  False  False  
Drug_Tar

In [170]:
doubleBaseTable = generateBooleanDoubleBaseTable(baseTable)


In [171]:
doubleBaseTable.columns

MultiIndex(levels=[[(u'A', u'A'), (u'A', u'C'), (u'A', u'G'), (u'A', u'T'), (u'C', u'A'), (u'C', u'C'), (u'C', u'G'), (u'C', u'T'), (u'G', u'A'), (u'G', u'C'), (u'G', u'G'), (u'G', u'T'), (u'T', u'A'), (u'T', u'C'), (u'T', u'G'), (u'T', u'T')], [-30, -29, -28, -27, -26, -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7]],
           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8

In [175]:
cols = [z for z in zip(doubleBaseTable.columns.get_level_values(0),
                                 doubleBaseTable.columns.get_level_values(1))]


In [176]:
cols

[(('A', 'A'), -30),
 (('A', 'A'), -29),
 (('A', 'A'), -28),
 (('A', 'A'), -27),
 (('A', 'A'), -26),
 (('A', 'A'), -25),
 (('A', 'A'), -24),
 (('A', 'A'), -23),
 (('A', 'A'), -22),
 (('A', 'A'), -21),
 (('A', 'A'), -20),
 (('A', 'A'), -19),
 (('A', 'A'), -18),
 (('A', 'A'), -17),
 (('A', 'A'), -16),
 (('A', 'A'), -15),
 (('A', 'A'), -14),
 (('A', 'A'), -13),
 (('A', 'A'), -12),
 (('A', 'A'), -11),
 (('A', 'A'), -10),
 (('A', 'A'), -9),
 (('A', 'A'), -8),
 (('A', 'A'), -7),
 (('A', 'A'), -6),
 (('A', 'A'), -5),
 (('A', 'A'), -4),
 (('A', 'A'), -3),
 (('A', 'A'), -2),
 (('A', 'A'), -1),
 (('A', 'A'), 0),
 (('A', 'A'), 1),
 (('A', 'A'), 2),
 (('A', 'A'), 3),
 (('A', 'A'), 4),
 (('A', 'A'), 5),
 (('A', 'A'), 6),
 (('A', 'A'), 7),
 (('A', 'G'), -30),
 (('A', 'G'), -29),
 (('A', 'G'), -28),
 (('A', 'G'), -27),
 (('A', 'G'), -26),
 (('A', 'G'), -25),
 (('A', 'G'), -24),
 (('A', 'G'), -23),
 (('A', 'G'), -22),
 (('A', 'G'), -21),
 (('A', 'G'), -20),
 (('A', 'G'), -19),
 (('A', 'G'), -18),
 (('A

In [177]:
# cols = [' '.join(z) for z in zip(["".join(x) for x in doubleBaseTable.columns.get_level_values(0)],
#                                                     [str(y) for y in doubleBaseTable.columns.get_level_values(1)])]
doubleBaseTable.columns = doubleBaseTable.columns.droplevel(level=1)
doubleBaseTable.columns = cols

In [178]:
doubleBaseTable.head()

,"((A, A), -30)","((A, A), -29)","((A, A), -28)","((A, A), -27)","((A, A), -26)","((A, A), -25)","((A, A), -24)","((A, A), -23)","((A, A), -22)","((A, A), -21)",...,"((T, T), -2)","((T, T), -1)","((T, T), 0)","((T, T), 1)","((T, T), 2)","((T, T), 3)","((T, T), 4)","((T, T), 5)","((T, T), 6)","((T, T), 7)"
sgId,,,,,,,,,,,,,,,,,,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [25]:
baseList = ['A','G','C','T']
homopolymerTable = pd.concat([libraryTable.apply(lambda row: np.floor(getMaxLengthHomopolymer(row['sequence'], base)), axis=1) for base in baseList],keys=baseList,axis=1)



In [27]:
homopolymerTable.head()

,A,G,C,T
sgId,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,1.0,2.0,4.0,1.0
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,1.0,2.0,3.0,1.0
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,1.0,2.0,3.0,1.0
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,1.0,3.0,3.0,2.0
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,1.0,2.0,1.0,2.0


In [26]:
baseFractions = pd.concat([libraryTable.apply(lambda row: getFractionBaseList(row['sequence'], ['A']),axis=1),
                        libraryTable.apply(lambda row: getFractionBaseList(row['sequence'], ['G']),axis=1),
                        libraryTable.apply(lambda row: getFractionBaseList(row['sequence'], ['C']),axis=1),
                        libraryTable.apply(lambda row: getFractionBaseList(row['sequence'], ['T']),axis=1),
                        libraryTable.apply(lambda row: getFractionBaseList(row['sequence'], ['G','C']),axis=1),
                        libraryTable.apply(lambda row: getFractionBaseList(row['sequence'], ['G','A']),axis=1),
                        libraryTable.apply(lambda row: getFractionBaseList(row['sequence'], ['C','A']),axis=1)],keys=['A','G','C','T','GC','purine','CA'],axis=1)


In [28]:
baseFractions.head()

,A,G,C,T,GC,purine,CA
sgId,,,,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,0.142857,0.333333,0.476190,0.047619,0.809524,0.476190,0.619048
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,0.095238,0.380952,0.380952,0.142857,0.761905,0.476190,0.476190
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,0.238095,0.428571,0.238095,0.095238,0.666667,0.666667,0.476190
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,0.047619,0.523810,0.285714,0.142857,0.809524,0.571429,0.333333
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,0.142857,0.428571,0.238095,0.190476,0.666667,0.571429,0.380952


In [34]:
dnaseSeries = getChromatinDataSeriesByGene(bwFileHandleDict['dnase'], libraryTable, sgInfoTable, p1p2Table, sgrnaPositionTable_p1p2)


In [37]:
dnaseSeries.head()

sgId
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1    0.047101
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1    0.038820
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1    0.131211
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1    0.239001
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1    0.190864
dtype: float64

In [35]:
faireSeries = getChromatinDataSeriesByGene(bwFileHandleDict['faire'], libraryTable, sgInfoTable, p1p2Table, sgrnaPositionTable_p1p2)


In [38]:
faireSeries.head()

sgId
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1    0.297187
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1    0.350605
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1    0.391916
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1    0.491987
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1    0.812678
dtype: float64

In [36]:
mnaseSeries = getChromatinDataSeriesByGene(bwFileHandleDict['mnase'], libraryTable, sgInfoTable, p1p2Table, sgrnaPositionTable_p1p2)


In [39]:
mnaseSeries.head()

sgId
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1    0.059896
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1    0.101563
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1    0.023438
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1    0.033854
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1    0.079427
dtype: float64

In [40]:
rnafolding = getRNAfoldingTable(libraryTable)


In [41]:
rnafolding.head()

no scaffold  \
                                                   RNA minimum free energy   
sgId                                                                         
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...                    -2.1   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...                    -2.6   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...                    -1.2   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...                    -3.1   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...                    -9.7   

                                                             with scaffold  \
                                                   RNA minimum free energy   
sgId                                                                         
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...                   -34.7   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...                   -36.3   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...                   -33.6   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...                   -39.6   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...                   -37.1   

                                                   is Paired                \
                                                         -20    -19    -18   
sgId                                                                         
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...     False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...      True  False   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...     False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...      True   True  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...      True   True   True   

                                                                         \
                                                      -17    -16    -15   
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...   True  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...   True   True  False   

                                                                         \
                                                      -14    -13    -12   
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...  False   True   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...   True   True   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...  False   True   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...  False  False  False   

                                                                         \
                                                      -11    -10     -9   
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...   True   True   True   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...  False   True  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362...   True   True  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441...  False  False   True   

                                                                         \
                                                       -8     -7     -6   
sgId                                                                      
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216...  False  False  False   
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296...   True   True   True   
Drug_Targe

In [43]:
sgrnaPositionTable_p1p2.iloc[:,2:].head()

,primary TSS-Up,primary TSS-Down,secondary TSS-Up,secondary TSS-Down
sgId,,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,205,167,247,206
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,125,87,167,126
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,103,65,145,104
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,59,21,101,60
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,-20,-58,22,-19


In [53]:
rnafolding.index

Index([u'Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_+_70323466.24-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_-_70323225.22-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_-_70323276.23-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_-_70323332.23-all~e39m1',
       u'Drug_Targets+Kinase_Phosphatase=AARS_-_70323472.23-all~e39m1',
       ...
       u'Apoptosis+Cancer+Other_Cancer=ZWINT_+_58120881.23-all~e39m1',
       u'Apoptosis+Cancer+Other_Cancer=ZWINT_+_58120902.24-all~e39m1',
       u'Apoptosis+Cancer+Other_Cancer=ZWINT_+_58120920.24-all~e39m1',
       u'Apoptosis+Cancer+Other_Cancer=ZWINT_+_58121021.

In [68]:
df1 = pd.concat([dnaseSeries,faireSeries,mnaseSeries],keys=['DNase','FAIRE','MNase'], axis=1)
df2 = pd.concat([dnaseSeries,faireSeries,mnaseSeries],keys=['DNase','FAIRE','MNase'], axis=1, sort=False)

all(df1.index == df2.index)

True

In [70]:
df1

,DNase,FAIRE,MNase
sgId,,,
Drug_Targets+Kinase_Phosphatase=AARS_+_70323216.24-all~e39m1,0.047101,0.297187,0.059896
Drug_Targets+Kinase_Phosphatase=AARS_+_70323296.24-all~e39m1,0.038820,0.350605,0.101563
Drug_Targets+Kinase_Phosphatase=AARS_+_70323318.24-all~e39m1,0.131211,0.391916,0.023438
Drug_Targets+Kinase_Phosphatase=AARS_+_70323362.24-all~e39m1,0.239001,0.491987,0.033854
Drug_Targets+Kinase_Phosphatase=AARS_+_70323441.24-all~e39m1,0.190864,0.812678,0.079427
Drug_Targets+Kinase_Phosphatase=AARS_+_70323466.24-all~e39m1,0.622671,0.886040,0.065104
Drug_Targets+Kinase_Phosphatase=AARS_-_70323225.22-all~e39m1,0.039667,0.293124,0.031250
Drug_Targets+Kinase_Phosphatase=AARS_-_70323276.23-all~e39m1,0.048474,0.321442,0.054348
Drug_Targets+Kinase_Phosphatase=AARS_-_70323332.23-all~e39m1,0.072239,0.370866,0.057065


In [78]:
epigenomic = pd.concat([dnaseSeries,faireSeries,mnaseSeries],keys=['DNase','FAIRE','MNase'], axis=1)


In [181]:


df_dict = {
    'length': lengthSeries,
    'distance': sgrnaPositionTable_p1p2.iloc[:,2:],
    'homopolymers': homopolymerTable,
    'base fractions': baseFractions,
    'strand': strand,
    'base table-A': booleanBaseTable['A'],
    'base table-T': booleanBaseTable['T'],
    'base table-G': booleanBaseTable['G'],
    'base table-C': booleanBaseTable['C'],
    'base dimers': doubleBaseTable,
    'accessibility': epigenomic,
    'RNA folding-no scaffold': rnafolding['no scaffold'],
    'RNA folding-with scaffold': rnafolding['with scaffold'],
    'RNA folding-pairing no scaffold': rnafolding['is Paired']
}



In [182]:
_keys, _values = zip(*df_dict.items())
paramTable_trainingGuides = pd.concat(_values, axis=1, keys=_keys)


/Users/ucbbiosciences/software/anaconda2/envs/CRISPRiaDesign/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [183]:
paramTable_trainingGuides.head()

RNA folding-with scaffold  \
                                                     RNA minimum free energy   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306124.2...                     -35.0   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306178.2...                     -32.5   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306366.2...                     -32.0   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306169.2...                     -32.1   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306286.2...                     -35.7   

                                                   base fractions            \
                                                                A         G   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306124.2...       0.142857  0.523810   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306178.2...       0.300000  0.450000   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306366.2...       0.238095  0.238095   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306169.2...       0.238095  0.238095   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306286.2...       0.190476  0.428571   

                                                                        \
                                                           C         T   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306124.2...  0.238095  0.095238   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306178.2...  0.150000  0.100000   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306366.2...  0.380952  0.142857   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306169.2...  0.476190  0.047619   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306286.2...  0.238095  0.142857   

                                                                        \
                                                          GC    purine   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306124.2...  0.761905  0.666667   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306178.2...  0.600000  0.750000   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306366.2...  0.619048  0.476190   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306169.2...  0.714286  0.476190   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306286.2...  0.666667  0.619048   

                                                             accessibility  \
                                                          CA         DNase   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306124.2...  0.380952      0.327138   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306178.2...  0.450000      0.612413   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306366.2...  0.619048      0.083953   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306169.2...  0.714286      0.446561   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306286.2...  0.428571      0.608116   

                                                              ...  \
                                                       FAIRE  ...   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306124.2...  0.929107  ...   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306178.2...  0.973006  ...   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306366.2...  0.500987  ...   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306169.2...  0.955371  ...   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306286.2...  0.801145  ...   

                                                   base table-T                \
                                                              0      1      2   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306124.2...        False  False  False   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306178.2...        False  False  False   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306366.2...        False  False  False   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306169.2...        False  False   True   
Apoptosis+Cancer+Other_Cancer=AATF_-_35306286.2...        False  False  False   

                                                                         \
                                                        3      4      5   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306124.2...  False  False   True   
Apoptosis+Cancer+Other_Cancer=AATF_+_35306178.2...   True   True  False   

#### Fails:

In [10]:
paramTable_trainingGuides = generateTypicalParamTable(libraryTable_training,sgInfoTable_training, tssTable, p1p2Table, genomeDict, bwhandleDict)

....

<string>:619: RuntimeWarning: Mean of empty slice
<string>:595: RuntimeWarning: All-NaN slice encountered
<string>:619: RuntimeWarning: invalid value encountered in float_scalars


.Done!

<string>:686: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



AssertionError: Cannot concat indices that do not have the same number of levels

## Fit parameters

In [165]:
pd.__version__

u'0.24.2'

In [157]:
#load in the 5-fold cross-validation splits used to generate the model
import cPickle
with open('data_files/CRISPRi_trainingdata_traintestsets.txt') as infile:
    geneFold_train, geneFold_test, fitTable = pd.read_pickle(infile, compression=None)


In [167]:
fitTable.loc[('base fractions', 'A'),'params']

{'bin function': <function numpy.lib.function_base.median>,
 'bin width': 0.1,
 'min edge data': 50}

In [166]:
fitTable

type  \
length                           length                   binnable_onehot   
distance                         primary TSS-Up                continuous   
                                 primary TSS-Down              continuous   
                                 secondary TSS-Up              continuous   
                                 secondary TSS-Down            continuous   
homopolymers                     A                        binnable_onehot   
                                 G                        binnable_onehot   
                                 C                        binnable_onehot   
                                 T                        binnable_onehot   
base fractions                   A                        binnable_onehot   
                                 G                        binnable_onehot   
                                 C                        binnable_onehot   
                                 T                        binnable_onehot   
                                 GC                       binnable_onehot   
                                 purine                   binnable_onehot   
                                 CA                       binnable_onehot   
strand                           same strand                       binary   
base table-A                     -30                               binary   
                                 -29                               binary   
                                 -28                               binary   
                                 -27                               binary   
                                 -26                               binary   
                                 -25                               binary   
                                 -24                               binary   
                                 -23                               binary   
                                 -22                               binary   
                                 -21                               binary   
                                 -20                               binary   
                                 -19                               binary   
                                 -18                               binary   
...                                                                   ...   
base dimers                      (('T', 'T'), 1)                   binary   
                                 (('T', 'T'), 2)                   binary   
                                 (('T', 'T'), 3)                   binary   
                                 (('T', 'T'), 4)                   binary   
                                 (('T', 'T'), 5)                   binary   
                                 (('T', 'T'), 6)                   binary   
                                 (('T', 'T'), 7)                   binary   
accessibility                    DNase                    binnable_onehot   
                                 FAIRE                    binnable_onehot   
                                 MNase                    binnable_onehot   
RNA folding-no scaffold          RNA minimum free energy  binnable_onehot   
RNA folding-with scaffold        RNA minimum free energy  binnable_onehot   
RNA folding-pairing, no scaffold -20                               binary   
                                 -19                               binary   
                                 -18                               binary   
                                 -17                               binary   
                                 -16                               binary   
                                 -15                               binary   
                                 -14                               binary   
                                 -13                               binary   
                                 -12                             

### XXX WORKS UNTIL HERE XXX

In [189]:
transformedParams_train, estimators = fitParams(paramTable_trainingGuides.loc[activityScores.dropna().index].iloc[geneFold_train], 
                                                activityScores.loc[activityScores.dropna().index].iloc[geneFold_train], 
                                                fitTable)


{'C': 0.5, 'gamma': 0.0005}
{'C': 0.5, 'gamma': 0.0005}
{'C': 0.5, 'gamma': 0.0001}
{'C': 0.5, 'gamma': 0.0005}
((accessibility, DNase)
0.0     14761.0
1.0         4.0
2.0         1.0
3.0         2.0
4.0         1.0
6.0         1.0
7.0         1.0
10.0        1.0
11.0        1.0
Name: (accessibility, DNase), dtype: float64, 0, -10, 50, 1)


ValueError: min edge requirements cannot be met

In [ ]:
transformedParams_train

In [161]:

transformedParams_test = transformParams(paramTable_trainingGuides.loc[activityScores.dropna().index].iloc[geneFold_test], fitTable, estimators)

reg = linear_model.ElasticNetCV(l1_ratio=[.5, .75, .9, .99, 1], n_jobs=16, max_iter=2000)

scaler = preprocessing.StandardScaler()
reg.fit(scaler.fit_transform(transformedParams_train), activityScores.loc[activityScores.dropna().index].iloc[geneFold_train])
predictedScores = pd.Series(reg.predict(scaler.transform(transformedParams_test)), index=transformedParams_test.index)
testScores = activityScores.loc[activityScores.dropna().index].iloc[geneFold_test]

print 'Prediction AUC-ROC:', metrics.roc_auc_score((testScores >= .75).values, np.array(predictedScores.values,dtype='float64'))
print 'Prediction R^2:', reg.score(scaler.transform(transformedParams_test), testScores)
print 'Regression parameters:', reg.l1_ratio_, reg.alpha_
coefs = pd.DataFrame(zip(*[abs(reg.coef_),reg.coef_]), index = transformedParams_test.columns, columns=['abs','true'])
print 'Number of features used:', len(coefs) - sum(coefs['abs'] < .00000000001)

('base fractions', 'A') {'C': 0.5, 'gamma': 0.0005}


KeyboardInterrupt: 

In [ ]:
#can save state for reproducing estimators later
#the pickling of the scikit-learn estimators/regressors will allow the model to be reloaded for prediction of other guide designs, 
#   but will not be compatible across scikit-learn versions, so it is important to preserve the training data and training/test folds
import cPickle
estimatorString = cPickle.dumps((fitTable, estimators, scaler, reg))
with open(PICKLE_FILE,'w') as outfile:
    outfile.write(estimatorString)
    
#also save the transformed parameters as these can slightly differ based on the automated binning strategy
transformedParams_train.head().to_csv(TRANSFORMED_PARAM_HEADER,sep='\t')

# 2. Applying machine learning model to predict sgRNA activity

## Generate TSS annotation using FANTOM dataset

In [ ]:
#you can supply any table of gene transcription start sites formatted as below
#for demonstration purposes, the rest of this walkthrough will use a small arbitrary subset of the protein coding TSS table
tssTable_new = tssTable.iloc[10:20, :-1]
tssTable_new.head()

In [ ]:
#if desired, use the ensembl annotation and the HGNC database to supply gene aliases to assist P1P2 matching in the next step
gencodeData = loadGencodeData('large_data_files/gencode.v19.annotation.gtf')
geneToAliases = generateAliasDict('large_data_files/20150424_HGNC_symbols.txt',gencodeData)

In [ ]:
#Now create a TSS annotation by searching for P1 and P2 peaks near annotated TSSs
#same parameters as for our lncRNA libraries
p1p2Table_new = generateTssTable_P1P2strategy(tssTable_new, 'large_data_files/TSS_human.sorted.bed.gz', 
                                                  matchedp1p2Window = 30000, #region around supplied TSS annotation to search for a FANTOM P1 or P2 peak that matches the gene name (or alias)
                                                  anyp1p2Window = 500, #region around supplied TSS annotation to search for the nearest P1 or P2 peak
                                                  anyPeakWindow = 200, #region around supplied TSS annotation to search for any CAGE peak
                                                  minDistanceForTwoTSS = 1000, #If a P1 and P2 peak are found, maximum distance at which to combine into a single annotation (with primary/secondary TSS positions)
                                                  aliasDict = geneToAliases[0])
#the function will report some collisions of IDs due to use of aliases and redundancy in genome, but will resolve these itself

In [ ]:
p1p2Table_new.head()

In [ ]:
p1p2Table_new.groupby('TSS source').agg(len).iloc[:,[2]]

In [ ]:
len(p1p2Table_new)

In [ ]:
#save tables
tssTable_alllncs.to_csv(TSS_TABLE_PATH,sep='\t', index_col=range(2))
p1p2Table_alllncs.to_csv(P1P2_TABLE_PATH,sep='\t', header=0, index_col=range(2))

## Find all sgRNAs in genomic regions of interest 

In [ ]:
libraryTable_new, sgInfoTable_new = findAllGuides(p1p2Table_new, genomeDict, 
                                                  (-25,500)) #region around P1P2 TSSs to search for new sgRNAs; recommend -550,-25 for CRISPRa

In [ ]:
len(libraryTable_new)

In [ ]:
libraryTable_new.head()

## Predicting sgRNA activity

In [ ]:
#calculate parameters for new sgRNAs
paramTable_new = generateTypicalParamTable(libraryTable_new, sgInfoTable_new, tssTable_new, p1p2Table_new, genomeDict, bwhandleDict)

In [ ]:
paramTable_new.head()

In [ ]:
#if starting from a separate session from where you ran the sgRNA learning steps of Part 1, reload the following
import cPickle
with open(PICKLE_FILE) as infile:
    fitTable, estimators, scaler, reg = cPickle.load(infile)
    
transformedParams_train = pd.read_csv(TRANSFORMED_PARAM_HEADER,sep='\t')

In [ ]:
#transform and predict scores according to sgRNA prediction model
transformedParams_new = transformParams(paramTable_new, fitTable, estimators)

#reconcile any differences in column headers generated by automated binning
colTups = []
for (l1, l2), col in transformedParams_new.iteritems():
    colTups.append((l1,str(l2)))
transformedParams_new.columns = pd.MultiIndex.from_tuples(colTups)

predictedScores_new = pd.Series(reg.predict(scaler.transform(transformedParams_new.loc[:, transformedParams_train.columns].fillna(0).values)), index=transformedParams_new.index)


In [ ]:
predictedScores_new.head()

In [ ]:
libraryTable_new.to_csv(LIBRARY_TABLE_PATH,sep='\t')
sgInfoTable_new.to_csv(sgRNA_INFO_PATH,sep='\t')
predictedScores_new.to_csv(PREDICTED_SCORES_PATH, sep='\t')


# 3. Construct sgRNA libraries
## Score sgRNAs for off-target potential

### There are many ways to score sgRNAs as off-target; below is one listed one method that is simple and flexible, but ignores gapped alignments, alternate PAMs, and uses bowtie which may not be maximally sensitive in all cases

In [ ]:
!mkdir temp_bowtie_files

In [ ]:
#output all sequences to a temporary FASTQ file for running bowtie alignment
fqFile = 'temp_bowtie_files/bowtie_input.fq'

def outputTempBowtieFastq(libraryTable, outputFileName):
    phredString = 'I4!=======44444+++++++' #weighting for how impactful mismatches are along sgRNA sequence 
    with open(outputFileName,'w') as outfile:
        for name, row in libraryTable.iterrows():
            outfile.write('@' + name + '\n')
            outfile.write('CCN' + str(Seq.Seq(row['sequence'][1:]).reverse_complement()) + '\n')
            outfile.write('+\n')
            outfile.write(phredString + '\n')
            
outputTempBowtieFastq(libraryTable_new, fqFile)

In [ ]:
import subprocess

#specifying a list of parameters to run bowtie with
#each tuple contains
# *the mismatch threshold below which a site is considered a potential off-target (higher is more stringent)
# *the number of sites allowed (1 is minimum since each sgRNA should have one true site in genome)
# *the genome index against which to align the sgRNA sequences; these can be custom built to only consider sites near TSSs
# *a name for the bowtie run to create appropriately named output files
alignmentList = [(39,1,'large_data_files/hg19.ensemblTSSflank500b','39_nearTSS'),
                (31,1,'large_data_files/hg19.ensemblTSSflank500b','31_nearTSS'),
                (21,1,'large_data_files/hg19_maskChrMandPAR','21_genome'),
                (31,2,'large_data_files/hg19.ensemblTSSflank500b','31_2_nearTSS'),
                (31,3,'large_data_files/hg19.ensemblTSSflank500b','31_3_nearTSS')]

alignmentColumns = []
for btThreshold, mflag, bowtieIndex, runname in alignmentList:

    alignedFile = 'temp_bowtie_files/' + runname + '_aligned.txt'
    unalignedFile = 'temp_bowtie_files/' + runname + '_unaligned.fq'
    maxFile = 'temp_bowtie_files/' + runname + '_max.fq'
    
    bowtieString = 'bowtie -n 3 -l 15 -e '+str(btThreshold)+' -m ' + str(mflag) + ' --nomaqround -a --tryhard -p 16 --chunkmbs 256 ' + bowtieIndex + ' --suppress 5,6,7 --un ' + unalignedFile + ' --max ' + maxFile + ' '+ ' -q '+fqFile+' '+ alignedFile
    print bowtieString
    print subprocess.call(bowtieString, shell=True) #0 means finished without errors

    #parse through the file of sgRNAs that exceeded "m", the maximum allowable alignments, and mark "True" any that are found
    try:
        with open(maxFile) as infile:
            sgsAligning = set()
            for i, line in enumerate(infile):
                if i%4 == 0: #id line
                    sgsAligning.add(line.strip()[1:])
    except IOError: #no sgRNAs exceeded m, so no maxFile created
        sgsAligning = set()
                    
    alignmentColumns.append(libraryTable_new.apply(lambda row: row.name in sgsAligning, axis=1))
    
#collate results into a table, and flip the boolean values to yield the sgRNAs that passed filter as True
alignmentTable = pd.concat(alignmentColumns,axis=1, keys=zip(*alignmentList)[3]).ne(True)

In [ ]:
alignmentTable.head() #True = passed threshold

## Pick the top sgRNAs for a library, given predicted activity scores and off-target filtering

In [ ]:
#combine all generated data into one master table
predictedScores_new.name = 'predicted score'
v2Table = pd.concat((libraryTable_new, predictedScores_new, alignmentTable, sgInfoTable_new), axis=1, keys=['library table v2', 'predicted score', 'off-target filters', 'sgRNA info'])

In [ ]:
import re
#for our pCRISPRi/a-v2 vector, we append flanking sequences to each sgRNA sequence for cloning and require the oligo to contain
#exactly 1 BstXI and BlpI site each for cloning, and exactly 0 SbfI sites for sequencing sample preparation
restrictionSites = {re.compile('CCA......TGG'):1,
                   re.compile('GCT.AGC'):1,
                   re.compile('CCTGCAGG'):0}

def matchREsites(sequence, REdict):
    seq = sequence.upper()
    for resite, numMatchesExpected in restrictionSites.iteritems():
        if len(resite.findall(seq)) != numMatchesExpected:
            return False
        
    return True

def checkOverlaps(leftPosition, acceptedLeftPositions, nonoverlapMin):
    for pos in acceptedLeftPositions:
        if abs(pos - leftPosition) < nonoverlapMin:
            return False
    return True

In [ ]:
#flanking sequences
upstreamConstant = 'CCACCTTGTTG'
downstreamConstant = 'GTTTAAGAGCTAAGCTG'

#minimum overlap between two sgRNAs targeting the same TSS
nonoverlapMin = 3

#number of sgRNAs to pick per gene/TSS
sgRNAsToPick = 10

#list of off-target filter (or combinations of filters) levels, matching the names in the alignment table above
offTargetLevels = [['31_nearTSS', '21_genome'],
                  ['31_nearTSS'],
                  ['21_genome'],
                  ['31_2_nearTSS'],
                  ['31_3_nearTSS']]

#for each gene/TSS, go through each sgRNA in descending order of predicted score
#if an sgRNA passes the restriction site, overlap, and off-target filters, accept it into the library
#if the number of sgRNAs accepted is less than sgRNAsToPick, reduce off-target stringency by one and continue
v2Groups = v2Table.groupby([('library table v2','gene'),('library table v2','transcripts')])
newSgIds = []
unfinishedTss = []
for (gene, transcript), group in v2Groups:
    geneSgIds = []
    geneLeftPositions = []
    empiricalSgIds = dict()
    
    stringency = 0
    
    while len(geneSgIds) < sgRNAsToPick and stringency < len(offTargetLevels):
        for sgId_v2, row in group.sort_values(('predicted score','predicted score'), ascending=False).iterrows():
            oligoSeq = upstreamConstant + row[('library table v2','sequence')] + downstreamConstant
            leftPos = row[('sgRNA info', 'position')] - (23 if row[('sgRNA info', 'strand')] == '-' else 0)
            if len(geneSgIds) < sgRNAsToPick and row['off-target filters'].loc[offTargetLevels[stringency]].all() \
                and matchREsites(oligoSeq, restrictionSites) \
                and checkOverlaps(leftPos, geneLeftPositions, nonoverlapMin):
                geneSgIds.append((sgId_v2,
                                  gene,transcript,
                                  row[('library table v2','sequence')], oligoSeq,
                                  row[('predicted score','predicted score')], np.nan,
                                 stringency))
                geneLeftPositions.append(leftPos)
                
        stringency += 1
            
    if len(geneSgIds) < sgRNAsToPick:
        unfinishedTss.append((gene, transcript)) #if the number of accepted sgRNAs is still less than sgRNAsToPick, discard gene
    else:
        newSgIds.extend(geneSgIds)
        
libraryTable_complete = pd.DataFrame(newSgIds, columns = ['sgID', 'gene', 'transcript','protospacer sequence', 'oligo sequence',
 'predicted score', 'empirical score', 'off-target stringency']).set_index('sgID')

In [ ]:
print len(libraryTable_complete)

In [ ]:
#number of sgRNAs accepted at each stringency level
libraryTable_complete.groupby('off-target stringency').agg(len).iloc[:,0]

In [ ]:
#number of TSSs with fewer than required number of sgRNAs (and thus not included in the library)
print len(unfinishedTss)

In [ ]:
libraryTable_complete.head()

# 5. Design negative controls matching the base composition of the library

In [ ]:
#calcluate the base frequency at each position of the sgRNA, then generate random sequences weighted by this frequency
def getBaseFrequencies(libraryTable, baseConversion = {'G':0, 'C':1, 'T':2, 'A':3}):
    baseArray = np.zeros((len(libraryTable),20))

    for i, (index, seq) in enumerate(libraryTable['protospacer sequence'].iteritems()):
        for j, char in enumerate(seq.upper()):
            baseArray[i,j] = baseConversion[char]

    baseTable = pd.DataFrame(baseArray, index = libraryTable.index)
    
    baseFrequencies = baseTable.apply(lambda col: col.groupby(col).agg(len)).fillna(0) / len(baseTable)
    baseFrequencies.index = ['G','C','T','A']
    
    baseCumulativeFrequencies = baseFrequencies.copy()
    baseCumulativeFrequencies.loc['C'] = baseFrequencies.loc['G'] + baseFrequencies.loc['C']
    baseCumulativeFrequencies.loc['T'] = baseFrequencies.loc['G'] + baseFrequencies.loc['C'] + baseFrequencies.loc['T']
    baseCumulativeFrequencies.loc['A'] = baseFrequencies.loc['G'] + baseFrequencies.loc['C'] + baseFrequencies.loc['T'] + baseFrequencies.loc['A']

    return baseFrequencies, baseCumulativeFrequencies

def generateRandomSequence(baseCumulativeFrequencies):
    randArray = np.random.random(baseCumulativeFrequencies.shape[1])
    
    seq = []
    for i, col in baseCumulativeFrequencies.iteritems():
        for base, freq in col.iteritems():
            if randArray[i] < freq:
                seq.append(base)
                break
                
    return ''.join(seq)

In [ ]:
baseCumulativeFrequencies = getBaseFrequencies(libraryTable_complete)[1]
negList = []
numberToGenerate = 1000 #can generate many more; some will be filtered out by off-targets, and you can always select an arbitrary subset for inclusion into the library
for i in range(numberToGenerate):
    negList.append(generateRandomSequence(baseCumulativeFrequencies))
negTable = pd.DataFrame(negList, index=['non-targeting_' + str(i) for i in range(numberToGenerate)], columns = ['sequence'])

fqFile = 'temp_bowtie_files/bowtie_input_negs.fq'
outputTempBowtieFastq(negTable, fqFile)

In [ ]:
#similar to targeting sgRNA off-target scoring, but looking for sgRNAs with 0 alignments
alignmentList = [(31,1,'~/indices/hg19.ensemblTSSflank500b','31_nearTSS_negs'),
                (21,1,'~/indices/hg19_maskChrMandPAR','21_genome_negs')]

alignmentColumns = []
for btThreshold, mflag, bowtieIndex, runname in alignmentList:

    alignedFile = 'temp_bowtie_files/' + runname + '_aligned.txt'
    unalignedFile = 'temp_bowtie_files/' + runname + '_unaligned.fq'
    maxFile = 'temp_bowtie_files/' + runname + '_max.fq'
    
    bowtieString = 'bowtie -n 3 -l 15 -e '+str(btThreshold)+' -m ' + str(mflag) + ' --nomaqround -a --tryhard -p 16 --chunkmbs 256 ' + bowtieIndex + ' --suppress 5,6,7 --un ' + unalignedFile + ' --max ' + maxFile + ' '+ ' -q '+fqFile+' '+ alignedFile
    print bowtieString
    print subprocess.call(bowtieString, shell=True)

    #read unaligned file for negs, and then don't flip boolean of alignmentTable
    with open(unalignedFile) as infile:
        sgsAligning = set()
        for i, line in enumerate(infile):
            if i%4 == 0: #id line
                sgsAligning.add(line.strip()[1:])

    alignmentColumns.append(negTable.apply(lambda row: row.name in sgsAligning, axis=1))
    
alignmentTable = pd.concat(alignmentColumns,axis=1, keys=zip(*alignmentList)[3])
alignmentTable.head()

In [ ]:
acceptedNegList = []
negCount = 0
for i, (name, row) in enumerate(pd.concat((negTable,alignmentTable),axis=1, keys=['seq','alignment']).iterrows()):
    oligo = upstreamConstant + row['seq','sequence'] + downstreamConstant
    if row['alignment'].all() and matchREsites(oligo, restrictionSites):
        acceptedNegList.append(('non-targeting_%05d' % negCount, 'negative_control', 'na', row['seq','sequence'], oligo, 0))
        negCount += 1
        
acceptedNegs = pd.DataFrame(acceptedNegList, columns = ['sgId', 'gene', 'transcript', 'protospacer sequence', 'oligo sequence', 'off-target stringency']).set_index('sgId')

In [ ]:
acceptedNegs.head()

In [ ]:
libraryTable_complete.to_csv(LIBRARY_WITHOUT_NEGATIVES_PATH, sep='\t')
acceptedNegs.to_csv(NEGATIVE_CONTROLS_PATH,sep='\t')

# 6. Finalizing library design

* divide genes into sublibrary groups (if required)
* assign negative control sgRNAs to sublibrary groups; ~1-2% of the number of sgRNAs in the library is a good rule-of-thumb
* append PCR adapter sequences (~18bp) to each end of the oligo sequences to enable amplification of the oligo pool; each sublibary should have an orthogonal sequence so they can be cloned separately